In [1]:
import sys

sys.path.append("/users/tom/git/neon_blink_detection/")
sys.path.append("/users/tom/git/neon_blink_detection/src")

from training.helper import ClassifierParams, Results
from src.neon_blink_detector import get_params
of_params, pp_params, _ = get_params()
import numpy as np
from training.helper import get_augmentation_options
from src.helper import OfParams, PPParams, AugParams
from training.video_loader import video_loader
from training.datasets_loader import concatenate_all_samples, concatenate

clip_names = np.load("/users/tom/git/neon_blink_detection/clip_list.npy")

of_params = OfParams(n_layers=5, layer_interval=7, average=False, img_shape=(64, 64), grid_size=4, step_size=7, window_size=11, stop_steps=3)

aug_params_options = get_augmentation_options()
aug_params = aug_params_options[0]

rec = video_loader(of_params, aug_params)

rec.collect(clip_names, bg_ratio=1, augment=False, idx=0)

/cluster/anaconda3/envs/tom_py310/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


options 1

Loading clip: 1000-2022-12-14-09-43-56-0fcac6d3
Number of clips: 1

Loading clip: 1002-2022-12-14-11-43-58-23e05b8c
Number of clips: 1

Loading clip: 1004-2022-12-14-13-14-14-c8a509b9
Number of clips: 2

Loading clip: 1005-2022-12-14-15-07-31-ba8d94d5
Number of clips: 2

Loading clip: 1010-2022-12-15-13-27-31-f46dcdd8
Number of clips: 1

Loading clip: 1140-2023-01-12-13-15-56-2f0172d2


KeyboardInterrupt: 

In [4]:
from training.cnn import OpticalFlowCNN, OpticalFlowDataset
from training.run_one import train_cnn

classifier, scores = train_cnn(
            rec,
            clip_names,
            classifier_params=None,
            export_path=None,
            idx=0,
            augment_data=False,
            pp_params=pp_params,
        )

/users/tom/git/neon_blink_detection/training/cnn.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)
/users/tom/git/neon_blink_detection/training/cnn.py:138: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y, dtype=torch.long)


Epoch 1/100
Validation loss: 0.26984216694305413
Epoch 2/100
Validation loss: 0.23062998276666133
Epoch 3/100
Validation loss: 0.20695850293893014
Epoch 4/100
Validation loss: 0.1894970672055535
Epoch 5/100
Validation loss: 0.1795741290017201
Epoch 6/100
Validation loss: 0.17512992568259692
Epoch 7/100
Validation loss: 0.17496164998270736
Epoch 8/100
Validation loss: 0.16099902608832956
Epoch 9/100
Validation loss: 0.15243457471204064
Epoch 10/100
Validation loss: 0.15183160753145705
Epoch 11/100
Validation loss: 0.15173971292691946
Epoch 12/100
Validation loss: 0.1433104367885929
Epoch 13/100
Validation loss: 0.143464439011512
Epoch 14/100
Validation loss: 0.1430923905781042
Epoch 15/100
Validation loss: 0.141280961936734
Epoch 16/100
Validation loss: 0.13528128966254038
Epoch 17/100
Validation loss: 0.1268652409355897
Epoch 18/100
Validation loss: 0.1266434966674904
Epoch 19/100
Validation loss: 0.12917113724932835
Epoch 20/100
Validation loss: 0.12388510059864416
Epoch 21/100
Valida

In [118]:
features = concatenate(rec.all_features, clip_names)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names)
labels = samples_gt.labels

In [119]:
import torch

torch_features = torch.from_numpy(features).float().cuda()
cnn_features = classifier.predict(torch_features)

/users/tom/git/neon_blink_detection/training/cnn.py:136: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=torch.float32)


In [120]:
length = 50

indices = np.arange(0, features.shape[0])
all_indices = np.array([np.arange(index - length, index + length) for index in indices])
all_indices = np.clip(all_indices, 0, features.shape[0]-1)

all_cnn_features = np.array(cnn_features[all_indices, :].reshape(-1, 2*length * 3))

In [121]:
# import xgb classifi
from functions.classifiers import Classifier as XGB
from functions.pipeline import get_classifier_params

classifier_params = get_classifier_params()
xgb = XGB(classifier_params)
xgb.on_fit(features=all_cnn_features, labels=labels)

on_fit took 1 m 41 s


In [112]:
clip_names_test = [
    "2023-03-01_09-59-07-2ea49126",  # kai bike
    "2023-01-27_15-59-54-49a115d5",  # tom computer
    "2023-02-01_11-45-11-7621531e",  # kai computer
    "2023-01-27_16-10-14-a2a8cbe1",  # ryan discussing
    "2023-01-27_16-15-26-57802f75",  # tom walking
    "2023-01-27_16-24-04-eb4305b1",  # kai walking
    "2023-01-27_16-31-52-5f743ed0",  # moritz snowboarding
    "padel_tennis_neon_01-b922b245",  # mgg padel
    "padel_tennis_neon_03-2ded8f56",  # mgg partner padel
]

rec.collect(clip_names_test, bg_ratio=1, augment=False, idx=1)


Loading clip: 2023-03-01_09-59-07-2ea49126
Number of clips: 2

Loading clip: 2023-01-27_15-59-54-49a115d5
Number of clips: 1

Loading clip: 2023-02-01_11-45-11-7621531e
Number of clips: 1

Loading clip: 2023-01-27_16-10-14-a2a8cbe1
Number of clips: 1

Loading clip: 2023-01-27_16-15-26-57802f75
Number of clips: 1

Loading clip: 2023-01-27_16-24-04-eb4305b1
Number of clips: 1

Loading clip: 2023-01-27_16-31-52-5f743ed0
Number of clips: 1

Loading clip: padel_tennis_neon_01-b922b245
Number of clips: 1

Loading clip: padel_tennis_neon_03-2ded8f56
Number of clips: 1


In [122]:

features = concatenate(rec.all_features, clip_names_test)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names_test)
labels = samples_gt.labels

In [123]:
indices = np.arange(0, features.shape[0])
all_indices = np.array([np.arange(index - length, index + length) for index in indices])
all_indices = np.clip(all_indices, 0, features.shape[0]-1)

torch_features_test = torch.from_numpy(features).float().cuda()
cnn_features_test = classifier.predict(torch_features_test)
all_cnn_features_test = np.array(cnn_features_test[all_indices, :].reshape(-1, 2 * length * 3))

In [124]:
pred_labels = np.argmax(xgb.predict(all_cnn_features_test), axis=1)

In [2]:
from sklearn.metrics import f1_score, recall_score, precision_score

print(f1_score(labels, pred_labels, average="macro"))
print(recall_score(labels, pred_labels,  average="macro"))
print(precision_score(labels, pred_labels,  average="macro"))

NameError: name 'labels' is not defined

In [80]:
cnn_features_test.shape

(176021, 3)

In [ ]:
[0.94316394 0.82210183 0.80179752]
[0.97496918 0.77304153 0.72975878]
[0.91336824 0.87781123 0.88961677]